In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import json

class Book:

    def __init__(self, title, price, authors):
        self.title = title
        self.price = price
        self.authors = authors
        self.author = [author.strip() for author in authors.split(",")]

    def __str__(self):
        return f"Book({self.title}, {self.price}, {self.author})"

    def __repr__(self):
        return f"Book({self.title}, {self.price}, {self.author})"

    def to_dict(self):
        return {
            "title": self.title,
            "price": self.price,
            "author": self.author,
        }

    def to_list(self):
        return [self.title, self.price, self.author]

# 요구사항 1. book 크롤링한 데이터를 데이터베이스에 담기
url = 'https://paullab.co.kr/bookservice/'
response = requests.get(url)
response.encoding = 'utf-8'
html = response.text

soup = BeautifulSoup(html, 'html.parser')
book_details = soup.select('.book_detail')

books_data = []
for item in book_details:
    book_name = item.select('.book_name')[0].text.strip()
    book_price = int(item.select('.book_info')[0].text[3:].strip().replace(',', '').replace('무료', '0').replace('원', ''))
    book_author = item.select('.book_info')[1].text[3:].strip()
    books_data.append(Book(book_name, book_price, book_author))

print(books_data)

db = sqlite3.connect('books.db')
conn = db.cursor()

## 데이터베이스를 생성
conn.execute("CREATE TABLE book_crawler (id INTEGER, title TEXT, price INTEGER, author TEXT)")

## 데이터 삽입
for idx, item in enumerate(books_data, 1):
    conn.execute(f"INSERT INTO book_crawler VALUES ({idx}, '{item.title}', '{item.price}', '{item.authors}')")

db.commit()
db.close()

# 요구사항2. 데이터베이스를 읽습니다.
db = sqlite3.connect('books.db')
conn = db.cursor()

## 데이터 rows에 가져오기
rows = conn.execute("SELECT id, title, price, author FROM book_crawler")


# 요구사항3. 데이터베이스를 JSON 형태로 출력하기. 출력 이름은 output.json
rows_array = []
for row in rows:
    rows_array.append({
        'id': row[0],
        'title': row[1],
        'price': row[2],
        'author': row[3]
    })

with open('output.json', 'w', encoding='utf-8') as f:
    json.dump(rows_array, f, ensure_ascii=False, indent=4)

db.close()

[Book(메모혁신 Notion(노션) 활용 가이드, 7000, ['이호준', '이준호', '김혜원', '김유진', '차경림', '김진', '현지연', '정승한']), Book(이력서 작성 가이드, 10000, ['강혜진', '김유진', '김혜원', '김진', '이범재', '이호준', '원유선', '박누리', '차경림', '최원범', '한재현']), Book(제주코딩베이스캠프 Code Festival: Python 100제 1부, 0, ['이호준', '강민정', '김유진', '정윤하', '최원범']), Book(튜토리얼로 배우는 HTML&CSS, 4900, ['이호준', '박민균', '양지혜', '정민준', '박소영', '전유빈']), Book(코딩도장 튜토리얼로 배우는 Python 1편 object, 3000, ['이호준', '정민준', '배희성', '김덕주', '한승협']), Book(코딩도장 튜토리얼로 배우는 python 2편 제어문, 4900, ['이호준', '우지훈', '박소영', '허재영']), Book(코딩도장 튜토리얼로 배우는 Python 문제풀이, 0, ['이호준', '전유빈', '김태범', '한승협', '김혜원', '강민정']), Book(타노스의 건틀릿 알고리즘 With Python, 0, ['이호준', '한승협', '강민정', '신민욱']), Book(xlsxwriter 튜토리얼로 배우는 Python 엑셀 프로그래밍, 0, ['이호준', '강상권', '오재원', '전진환', '문형철', '이현창']), Book(러플 튜토리얼로 배우는 Python, 4900, ['이호준', '배희성', '양지혜', '박소영', '김수연', '정민준']), Book(인공지능을 활용한 업무자동화 With Google Developers Group JEJU, 0, ['이호준', '이준호', '현암', '윤현담', '김루비', '이동민', '유승엽', '박서윤', '김채린', '한승협', '안상우', '장성원', '전진환', '강민성', '조민구', '전창삼'])